# Блок загрузки данных и первичной настройки

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

загружаем предварительно обработанные в ноутбуке _thesis_preprocessing.ipynb_ данные

In [ ]:
clicks_df = pd.read_csv('/kaggle/input/thesis-preprocessed-data/clicks_processed.csv') # для Kaggle
buys_df = pd.read_csv('/kaggle/input/thesis-preprocessed-data/buys_processed.csv') # для Kaggle
# clicks_df = pd.read_csv('drive/MyDrive/Thesis/clicks_processed.csv') #для google colab
# buys_df = pd.read_csv('drive/MyDrive/Thesis/buys_processed.csv') #для google colab
clicks_df.category = clicks_df.category.astype('string')
clicks_df['timestamp'] = pd.to_datetime(clicks_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
buys_df['timestamp'] = pd.to_datetime(buys_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
clicks_df.drop('category', inplace=True, axis=1)

общая информация о данных

In [ ]:
clicks_df.info()
buys_df.info()

Отсортируем события в рамках сессий по времени

In [ ]:
clicks_df = clicks_df.sort_values(by=['session_id', 'timestamp'])
buys_df = buys_df.sort_values(by=['session_id', 'timestamp'])

Проведем анализ длины сессий (количества кликов в каждой сессии)

# Блок EDA

In [ ]:
session_lengths = clicks_df.groupby('session_id').size()
print("\nСтатистика по длинам сессий (клики):")
print(session_lengths.describe())

# Гистограмма распределения длины сессий
plt.figure(figsize=(10,5))
session_lengths.hist(bins=100)
plt.title("Распределение длины сессий")
plt.xlabel("Длина сессии (кол-во кликов)")
plt.ylabel("Частота")
#plt.xscale('log')  # Применение логарифмической шкалы
plt.show()

In [ ]:
session_lengths[session_lengths <= 25]

In [ ]:
session_lengths[session_lengths > 25]

In [ ]:
session_lengths[session_lengths >= 100]

Как можем заметить, количество сессий с кликами не более 25 за сессию равно __9211993__, что является очень большим значением по сравнению с количеством сессий с кликами более 25, а именно __37736__, а также количеством сессий с кликами более 100, которых 474, объясняется протяженность графика распределения длины сессий

Определение самых популярных товаров и частота их появления помогает понять, насколько доминирует небольшой набор топ-товаров. Часто товары с очень низкой частотой исключают (James et al., 2013)

Посмотрим на распределение частоты кликов по товарам среди 10к наиболее кликабельных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Считаем частоту кликов по товарам
item_freq = clicks_df['item_id'].value_counts()

# Построение гистограммы
plt.figure(figsize=(10, 5))
plt.hist(item_freq, bins=100, log=False, edgecolor='black')  # log=True для логарифмической шкалы

# Оформление графика
plt.title("Распределение популярности товаров (по частоте кликов)")
plt.xlabel("Частота кликов")
plt.ylabel("Количество товаров")
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

Наблюдаем сильное снижение кликов

Посмотрим на распределение частоты кликов по товарам среди 10к наименее кликабельных

In [ ]:
item_freq = clicks_df['item_id'].value_counts()
print("\nСтатистика по частоте товаров:")
print(item_freq.describe())

plt.figure(figsize=(10,5))
item_freq.tail(10000).plot(kind='bar')
plt.title("Топ-10000 товаров по частоте кликов с конца")
plt.xlabel("Item ID")
plt.ylabel("Частота")
plt.show()

Складывается впечатление, что после какого-то момента нужно обрезать данные, т.е. выкидывать те товары, на которые люди кликали меньше всего

In [ ]:
min_time = clicks_df['timestamp'].min()
max_time = clicks_df['timestamp'].max()
print(f"\nДанные о кликах собраны с {min_time} по {max_time}")

clicks_df['date'] = clicks_df['timestamp'].dt.date
daily_sessions = clicks_df.groupby('date')['session_id'].nunique()

plt.figure(figsize=(10,5))
daily_sessions.plot(kind='line')
plt.title("Количество уникальных сессий по дням")
plt.xlabel("Дата")
plt.ylabel("Число уникальных сессий")
plt.show()

In [ ]:
# посмотрим, сколько сессий имеют длину <2
short_sessions = (session_lengths < 2).sum()
print(f"\nСессий длиной <2: {short_sessions}, это {short_sessions/len(session_lengths)*100:.2f}%")

# Проверим редкие товары
rare_items_threshold = 5
rare_items_count = (item_freq < rare_items_threshold).sum()
print(f"Товаров с частотой <{rare_items_threshold}: {rare_items_count}, это {rare_items_count/len(item_freq)*100:.2f}%")

Анализ и принятие решений на основе EDA:
Фильтрация коротких сессий (<2 кликов):
Причина: Сессии с одним кликом предоставляют мало информации для построения рекомендаций и могут быть неинформативными или шумовыми.

Фильтрация редких товаров (<5 кликов):
Причина: Товары с низкой частотой могут быть нерелевантными, новыми или случайными, добавляя шум в данные и увеличивая разреженность.

Проведем анализ датафрейма _buys_df_

In [ ]:
# Анализ количества покупок
purchase_counts = buys_df.groupby('session_id').size()
print("\nСтатистика по количеству покупок в сессиях:")
print(purchase_counts.describe())

# Гистограмма распределения количества покупок в сессии
plt.figure(figsize=(10,5))
purchase_counts.hist(bins=50)
plt.title("Распределение количества покупок в сессиях")
plt.xlabel("Количество покупок")
plt.ylabel("Частота")
plt.show()

# Анализ распределения цен
plt.figure(figsize=(10,5))
buys_df['price'].hist(bins=50)
plt.title("Распределение цен товаров")
plt.xlabel("Цена")
plt.ylabel("Частота")
plt.show()
buys_df['price'].describe()

# Блок обработки данных

Проведем фильтрацию данных

In [ ]:
filtered_clicks = clicks_df.copy()

# Устанавливаем пороги
min_session_length = 2
min_item_freq = 5

# Цикл фильтрации
while True:
    initial_shape = filtered_clicks.shape

    # Фильтрация сессий
    session_lengths = filtered_clicks.groupby('session_id').size()
    valid_sessions = session_lengths[session_lengths >= min_session_length].index
    filtered_clicks = filtered_clicks[filtered_clicks['session_id'].isin(valid_sessions)]

    # Фильтрация товаров
    item_freq = filtered_clicks['item_id'].value_counts()
    valid_items = item_freq[item_freq >= min_item_freq].index
    filtered_clicks = filtered_clicks[filtered_clicks['item_id'].isin(valid_items)]

    # Проверка изменений
    if filtered_clicks.shape == initial_shape:
        break

print(f"Итоговая размерность: {filtered_clicks.shape}")

In [ ]:
# отфильтруем данные о покупках
buys = buys_df.copy()
buys = buys[buys.session_id.isin(filtered_clicks.session_id)]
buys = buys[buys.item_id.isin(filtered_clicks.item_id)]

In [ ]:
del clicks_df, buys_df

Теперь разобьем данные на train и test по времени

In [ ]:
# Сортируем сессии по времени их завершения
session_end_times = filtered_clicks.groupby("session_id")["timestamp"].max().sort_values()
session_end_times = session_end_times.iloc[-100000:]
# Разделяем сессии на train и test
split_ratio = 0.8
num_train_sessions = int(len(session_end_times) * split_ratio)

train_sessions = session_end_times.index[:num_train_sessions]
test_sessions = session_end_times.index[num_train_sessions:]

train_clicks = filtered_clicks[filtered_clicks["session_id"].isin(train_sessions)]
test_clicks = filtered_clicks[filtered_clicks["session_id"].isin(test_sessions)]
train_items = set(train_clicks['item_id'])
test_clicks = test_clicks[test_clicks['item_id'].isin(train_items)]

train_buys = buys[buys["session_id"].isin(train_sessions)]
test_buys = buys[buys["session_id"].isin(test_sessions)]
test_buys = test_buys[test_buys['item_id'].isin(train_items)]

print(f"Train clicks: {len(train_clicks)}")
print(f"Test clicks: {len(test_clicks)}")
print(f"Train buys: {len(train_buys)}")
print(f"Test buys: {len(test_buys)}")

In [ ]:
def print_data_statistics(clicks, name="Dataset"):
    """
    Выводит базовую статистику о кликах.
    """
    n_sessions = clicks['session_id'].nunique()
    n_items = clicks['item_id'].nunique()
    n_interactions = len(clicks)

    print(f"{name}:")
    print(f"- Количество сессий: {n_sessions}")
    print(f"- Количество товаров: {n_items}")
    print(f"- Количество взаимодействий: {n_interactions}")


In [ ]:
print_data_statistics(train_clicks, "Исходные Train данные")
print_data_statistics(test_clicks, "Исходные Test данные")
print_data_statistics(train_buys, "Исходные Train данные")
print_data_statistics(test_buys, "Исходные Test данные")

In [ ]:
# описательные статистики цен и количества товаров
print(test_buys[["price", "quantity"]].describe())

In [ ]:
def check_distribution(clicks, name="Dataset"):
    session_lengths = clicks.groupby('session_id').size()
    print(f"{name}: Средняя длина сессии: {session_lengths.mean():.2f}, Медианная длина: {session_lengths.median():.2f}")

check_distribution(train_clicks, "Train")
check_distribution(test_clicks, "Test")


In [ ]:
# Убедимся, что все товары из теста есть в трейне
train_items = set(train_clicks['item_id'])
test_items = set(test_clicks['item_id'])

assert test_items.issubset(train_items), "Есть товары из теста, которые отсутствуют в трейне!"

In [ ]:
def sort_sessions_by_last_action(df):
    # Находим время последнего действия для каждой сессии
    df['last_action_time'] = df.groupby('session_id')['timestamp'].transform('max')

    # Сортируем по времени последнего действия, затем по session_id и timestamp
    sorted_df = df.sort_values(by=['last_action_time', 'session_id', 'timestamp']).reset_index(drop=True)

    # Удаляем временный столбец, если он не нужен
    sorted_df = sorted_df.drop(columns=['last_action_time'])

    return sorted_df

train_clicks_filtered = sort_sessions_by_last_action(train_clicks)
test_clicks_filtered = sort_sessions_by_last_action(test_clicks)
test_buys_filtered = sort_sessions_by_last_action(test_buys)

In [ ]:
display(train_clicks_filtered)
display(test_clicks_filtered)
display(test_buys_filtered)

In [ ]:
def split_test_sessions_optimized_v2(test_clicks, min_context=1, split_ratio=0.8):
    """
    Оптимизированное разбиение тестовых сессий на контекст и ground truth
    Убирает сессии, где контекст или ground truth пустые.
    """
    # Группируем данные по session_id
    grouped = test_clicks.groupby("session_id")["item_id"].apply(list).reset_index()
    grouped.columns = ["session_id", "items"]

    # Функция для разбиения одной сессии
    def split_session(session):
        items = session["items"]
        if len(items) <= min_context:
            return None  # Слишком короткая сессия, пропускаем
        split_index = max(min_context, int(len(items) * split_ratio))
        context = items[:split_index]
        ground_truth = items[split_index:]
        return (session["session_id"], context, ground_truth) if context and ground_truth else None

    # Применяем разбиение ко всем сессиям
    split_sessions = grouped.apply(split_session, axis=1)

    # Убираем None и возвращаем результат
    test_sessions_split = [s for s in split_sessions if s is not None]
    return test_sessions_split

In [ ]:
# Разбиваем тестовые сессии
test_sessions_split = split_test_sessions_optimized_v2(test_clicks_filtered, min_context=2, split_ratio=0.8)

# Проверяем результаты
print(f"Количество тестовых сессий после разбиения: {len(test_sessions_split)}")
print("Пример разбитых сессий:")
for session_id, context, ground_truth in test_sessions_split[:5]:
    print(f"Session ID: {session_id}, Context: {context}, Ground Truth: {ground_truth}")

In [ ]:
def check_split_statistics(test_sessions_split):
    context_lengths = [len(context) for _, context, _ in test_sessions_split]
    ground_truth_lengths = [len(ground_truth) for _, _, ground_truth in test_sessions_split]

    print("Длина контекста:")
    print(f"- Средняя: {np.mean(context_lengths):.2f}, Медиана: {np.median(context_lengths):.2f}")
    print(f"- Мин: {min(context_lengths)}, Макс: {max(context_lengths)}")

    print("\nДлина Ground Truth:")
    print(f"- Средняя: {np.mean(ground_truth_lengths):.2f}, Медиана: {np.median(ground_truth_lengths):.2f}")
    print(f"- Мин: {min(ground_truth_lengths)}, Макс: {max(ground_truth_lengths)}")

check_split_statistics(test_sessions_split)

# Блок базовых рекомендаций

Случайные рекомендации

In [ ]:
import random

def recommend_random_items(train_clicks, context, top_k=5, seed=42):
    """
    Генерирует случайные рекомендации на основе train данных.

    train_clicks: DataFrame с кликами в train.
    context: список товаров (контекст для текущей сессии, не используется в случайных рекомендациях).
    top_k: количество рекомендаций.
    seed: фиксированный seed для воспроизводимости.
    """
    if seed is not None:
        random.seed(seed)

    all_items = train_clicks["item_id"].unique()
    recommendations = random.sample(list(all_items), top_k)
    return recommendations

Популярные товары

In [ ]:
def recommend_popular_items(train_clicks, k=10):
    """
    Формирует рекомендации из самых популярных товаров.

    train_clicks: DataFrame с кликами в train.
    k: количество топ-N рекомендаций.
    """
    # Считаем популярность товаров (по количеству кликов)
    popular_items = train_clicks["item_id"].value_counts().head(k).index.tolist()
    return popular_items

Метрики

In [ ]:
def calculate_precision_recall(recommended_items, ground_truth, k):
    recommended_top_k = set(recommended_items[:k])
    ground_truth_set = set(ground_truth)
    intersection_size = len(recommended_top_k & ground_truth_set)

    precision = intersection_size / k if k > 0 else 0
    recall = intersection_size / len(ground_truth_set) if ground_truth_set else 0

    return precision, recall


def calculate_ndcg(recommended_items, ground_truth, k):
    recommended_top_k = recommended_items[:k]
    ground_truth_set = set(ground_truth)

    dcg = sum(
        [1 / np.log2(i + 2) for i, item in enumerate(recommended_top_k) if item in ground_truth_set]
    )

    idcg = sum(
        [1 / np.log2(i + 2) for i in range(min(len(ground_truth_set), k))]
    )

    return dcg / idcg if idcg > 0 else 0


def calculate_revenue_and_conversion(recommended_items, ground_truth_buys, test_buys, k):
    """
    Рассчитывает Revenue@k и Conversion@k для одной сессии.

    recommended_items: список рекомендованных товаров.
    ground_truth_buys: список товаров, купленных в ground truth.
    test_buys: DataFrame с данными о покупках в тесте.
    k: количество топ-N рекомендаций.
    """
    recommended_top_k = set(recommended_items[:k])
    ground_truth_buys_set = set(ground_truth_buys)

    # Revenue@k
    relevant_buys = test_buys[
        test_buys["item_id"].isin(recommended_top_k & ground_truth_buys_set)
    ]
    revenue = (relevant_buys["price"] * relevant_buys["quantity"]).sum()

    # Conversion@k
    conversion = int(bool(recommended_top_k & ground_truth_buys_set))

    return revenue, conversion


In [ ]:
def generate_random_recommendations(all_items, k, seed=None):
    if seed is not None:
        random.seed(seed)
    return random.sample(list(all_items), k)


In [ ]:
def evaluate_random_recommendations_optimized(train_clicks, test_sessions_split, test_buys, k=5, seed=None):
    """
    Оптимизированная оценка случайных рекомендаций для тестовых сессий.

    train_clicks: DataFrame с кликами в train.
    test_sessions_split: список тестовых сессий (SessionId, context, ground_truth).
    test_buys: DataFrame с покупками в тесте.
    k: количество рекомендаций.
    seed: фиксированный seed для воспроизводимости.
    """
    if seed is not None:
        random.seed(seed)

    # Все уникальные товары
    all_items = train_clicks["item_id"].unique()

    # Группируем test_buys по session_id
    test_buys_grouped = test_buys.groupby("session_id").agg(
        item_id=("item_id", list),
        price=("price", list),
        quantity=("quantity", list)
    ).to_dict(orient="index")

    metrics = {"precision": [], "recall": [], "ndcg": [], "revenue": [], "conversion": []}

    # Обрабатываем каждую сессию
    for session_id, context, ground_truth in test_sessions_split:
        # Генерируем случайные рекомендации
        random_recommendations = generate_random_recommendations(set(all_items) - set(context), k, seed)

        # Получаем покупки для текущей сессии
        session_buys = test_buys_grouped.get(session_id, {"item_id": [], "price": [], "quantity": []})
        ground_truth_buys = session_buys["item_id"]

        # Технические метрики
        precision, recall = calculate_precision_recall(random_recommendations, ground_truth, k)
        ndcg = calculate_ndcg(random_recommendations, ground_truth, k)

        # Бизнес-метрики
        revenue, conversion = calculate_revenue_and_conversion(random_recommendations, ground_truth_buys, test_buys, k)

        # Сохраняем метрики
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["ndcg"].append(ndcg)
        metrics["revenue"].append(revenue)
        metrics["conversion"].append(conversion)

    # Усредняем метрики
    average_metrics = {metric: np.mean(values) for metric, values in metrics.items()}
    return average_metrics


In [ ]:
# Оцениваем случайные рекомендации
average_metrics_random = evaluate_random_recommendations_optimized(train_clicks_filtered, test_sessions_split, test_buys_filtered, k=3, seed=42)
print("Средние метрики для случайных рекомендаций k=3:")
print(average_metrics_random)

average_metrics_random = evaluate_random_recommendations_optimized(train_clicks_filtered, test_sessions_split, test_buys_filtered, k=5, seed=42)
print("Средние метрики для случайных рекомендаций k=5:")
print(average_metrics_random)

average_metrics_random = evaluate_random_recommendations_optimized(train_clicks_filtered, test_sessions_split, test_buys_filtered, k=10, seed=42)
print("Средние метрики для случайных рекомендаций k=10:")
print(average_metrics_random)

In [ ]:
def evaluate_popular_recommendations(train_clicks, test_sessions_split, test_buys, k=10):
    """
    Оценивает рекомендации популярных товаров.

    train_clicks: DataFrame с кликами в train.
    test_sessions_split: список тестовых сессий (SessionId, context, ground_truth).
    test_buys: DataFrame с покупками в тесте.
    k: количество рекомендаций.
    """
    # Формируем список популярных товаров
    popular_recommendations = recommend_popular_items(train_clicks, k=k)

    metrics = {"precision": [], "recall": [], "ndcg": [], "revenue": [], "conversion": []}

    # Оцениваем рекомендации
    for session_id, context, ground_truth in test_sessions_split:
        # Получаем покупки для текущей сессии
        ground_truth_buys = test_buys[test_buys["session_id"] == session_id]["item_id"].tolist()

        # Технические метрики
        precision, recall = calculate_precision_recall(popular_recommendations, ground_truth, k)
        ndcg = calculate_ndcg(popular_recommendations, ground_truth, k)

        # Бизнес-метрики
        revenue, conversion = calculate_revenue_and_conversion(popular_recommendations, ground_truth_buys, test_buys, k)

        # Сохраняем метрики
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["ndcg"].append(ndcg)
        metrics["revenue"].append(revenue)
        metrics["conversion"].append(conversion)

    # Усредняем метрики
    average_metrics = {metric: np.mean(values) for metric, values in metrics.items()}
    return average_metrics

In [ ]:
# Оцениваем популярные товары
average_metrics_popular = evaluate_popular_recommendations(train_clicks_filtered, test_sessions_split, test_buys_filtered, k=3)
print("Средние метрики для популярных товаров k=3:")
print(average_metrics_popular)

average_metrics_popular = evaluate_popular_recommendations(train_clicks_filtered, test_sessions_split, test_buys_filtered, k=5)
print("Средние метрики для популярных товаров k=5:")
print(average_metrics_popular)

average_metrics_popular = evaluate_popular_recommendations(train_clicks_filtered, test_sessions_split, test_buys_filtered, k=10)
print("Средние метрики для популярных товаров k=10:")
print(average_metrics_popular)

# Блок более сложных алгоритмов (S-KNN и GRU4Rec)

метрики

In [ ]:
def calculate_precision_recall(recommended_items, ground_truth, k):
    recommended_top_k = set(recommended_items[:k])
    ground_truth_set = set(ground_truth)
    intersection_size = len(recommended_top_k & ground_truth_set)

    precision = intersection_size / k if k > 0 else 0
    recall = intersection_size / len(ground_truth_set) if ground_truth_set else 0

    return precision, recall


def calculate_ndcg(recommended_items, ground_truth, k):
    recommended_top_k = recommended_items[:k]
    ground_truth_set = set(ground_truth)

    dcg = sum(
        [1 / np.log2(i + 2) for i, item in enumerate(recommended_top_k) if item in ground_truth_set]
    )

    idcg = sum(
        [1 / np.log2(i + 2) for i in range(min(len(ground_truth_set), k))]
    )

    return dcg / idcg if idcg > 0 else 0


def calculate_revenue_and_conversion(recommended_items, ground_truth_buys, test_buys, k):
    """
    Рассчитывает Revenue@k и Conversion@k для одной сессии.

    recommended_items: список рекомендованных товаров.
    ground_truth_buys: список товаров, купленных в ground truth.
    test_buys: DataFrame с данными о покупках в тесте.
    k: количество топ-N рекомендаций.
    """
    recommended_top_k = set(recommended_items[:k])
    ground_truth_buys_set = set(ground_truth_buys)

    # Revenue@k
    relevant_buys = test_buys[
        test_buys["item_id"].isin(recommended_top_k & ground_truth_buys_set)
    ]
    revenue = (relevant_buys["price"] * relevant_buys["quantity"]).sum()

    # Conversion@k
    conversion = int(bool(recommended_top_k & ground_truth_buys_set))

    return revenue, conversion

In [ ]:
pip install pynndescent

In [ ]:
from scipy.sparse import csr_matrix

def prepare_sparse_interaction_matrix(train_clicks):
    """
    Создаёт разреженную матрицу взаимодействий session × item и маппинг session_id → index.
    """
    session_ids = train_clicks['session_id'].astype("category").cat.codes
    item_ids = train_clicks['item_id'].astype("category").cat.codes

    sparse_matrix = csr_matrix(
        (np.ones(len(train_clicks)), (session_ids, item_ids))
    )

    session_mapping = dict(enumerate(train_clicks['session_id'].astype("category").cat.categories))
    item_mapping = dict(enumerate(train_clicks['item_id'].astype("category").cat.categories))

    return sparse_matrix, session_mapping, item_mapping


# Создаём разреженную матрицу
sparse_matrix, session_mapping, item_mapping = prepare_sparse_interaction_matrix(train_clicks_filtered)
reverse_item_mapping = {item: idx for idx, item in item_mapping.items()}
print(f"Размер разреженной матрицы: {sparse_matrix.shape}")

In [ ]:
def get_neighbors_from_context(context, nn_descent_index, sparse_matrix, reverse_item_mapping, k_neighbors=5):
    """
    Получает соседей для тестовой сессии на основе её контекста.
    """
    # Создаём списки для индексов и значений
    indices = [reverse_item_mapping[item] for item in context if item in reverse_item_mapping]
    values = [1] * len(indices)

    # Создаём разреженный вектор взаимодействий
    context_vector = csr_matrix((values, ([0] * len(indices), indices)), shape=(1, sparse_matrix.shape[1]))

    # Поиск соседей
    neighbors, distances = nn_descent_index.query(context_vector, k=k_neighbors)
    return neighbors[0], distances[0]


In [ ]:
from pynndescent import NNDescent

# Создание индекса
nn_descent_index = NNDescent(
    sparse_matrix,
    metric="jaccard",  # Используем расстояние Жаккара
    n_neighbors=5,     # Количество ближайших соседей
    random_state=42
)

In [ ]:
import numpy as np
def recommend_items_sknn(sparse_matrix, neighbors, neighbor_distances, item_mapping, top_k=10):
    """
    Генерирует рекомендации на основе S-KNN.

    Parameters:
    - sparse_matrix: csr_matrix, матрица взаимодействий session × item.
    - neighbors: список индексов ближайших соседей.
    - neighbor_distances: расстояния до соседей.
    - item_mapping: словарь, сопоставляющий индексы товаров с их идентификаторами.
    - top_k: количество рекомендаций.

    Returns:
    - recommended_items: список рекомендованных товаров.
    """
    # Веса соседей на основе расстояний
    weights = 1 / (np.array(neighbor_distances) + 1e-10)  # Избегаем деления на 0
    weights_normalized = weights / np.sum(weights)

    # Учитываем веса соседей
    neighbor_items = sparse_matrix[neighbors].multiply(weights_normalized[:, np.newaxis]).sum(axis=0)

    # Преобразуем в одномерный массив и сортируем по убыванию
    recommended_indices = np.argsort(-neighbor_items.A1)[:top_k]
    recommended_items = [item_mapping[idx] for idx in recommended_indices]
    return recommended_items


In [ ]:
def evaluate_sknn_sparse(sparse_matrix, test_sessions_split, test_buys, item_mapping, reverse_item_mapping, nn_index, k_neighbors=5, top_k=10):
    """
    Оценка S-KNN на разреженной матрице с использованием NNDescent.
    """
    metrics = {"precision": [], "recall": [], "ndcg": [], "revenue": [], "conversion": []}

    for session_id, context, ground_truth in test_sessions_split:
        # Получаем соседей для контекста
        neighbors, distances = get_neighbors_from_context(context, nn_index, sparse_matrix, reverse_item_mapping, k_neighbors)

        # Генерируем рекомендации
        recommendations = recommend_items_sknn(sparse_matrix, neighbors, distances, item_mapping, top_k)

        # Получаем покупки для текущей сессии
        session_buys = test_buys[test_buys["session_id"] == session_id]
        ground_truth_buys = session_buys["item_id"].tolist()

        # Технические метрики
        precision, recall = calculate_precision_recall(recommendations, ground_truth, top_k)
        ndcg = calculate_ndcg(recommendations, ground_truth, top_k)

        # Бизнес-метрики
        revenue, conversion = calculate_revenue_and_conversion(recommendations, ground_truth_buys, test_buys, top_k)

        # Сохраняем метрики
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["ndcg"].append(ndcg)
        metrics["revenue"].append(revenue)
        metrics["conversion"].append(conversion)

    # Усредняем метрики
    average_metrics = {metric: np.mean(values) for metric, values in metrics.items()}
    return average_metrics

In [ ]:
average_metrics_sknn = evaluate_sknn_sparse(
    sparse_matrix,
    test_sessions_split,
    test_buys_filtered,
    item_mapping,
    reverse_item_mapping,
    nn_descent_index,
    k_neighbors=5,
    top_k=3
)
print("Средние метрики для S-KNN k=3:")
print(average_metrics_sknn)

average_metrics_sknn = evaluate_sknn_sparse(
    sparse_matrix,
    test_sessions_split,
    test_buys_filtered,
    item_mapping,
    reverse_item_mapping,
    nn_descent_index,
    k_neighbors=5,
    top_k=5
)
print("Средние метрики для S-KNN k=5:")
print(average_metrics_sknn)

average_metrics_sknn = evaluate_sknn_sparse(
    sparse_matrix,
    test_sessions_split,
    test_buys_filtered,
    item_mapping,
    reverse_item_mapping,
    nn_descent_index,
    k_neighbors=5,
    top_k=10
)
print("Средние метрики для S-KNN k=10:")
print(average_metrics_sknn)

---

In [ ]:
# Импорты
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Masking, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import random
from tqdm import tqdm

# Убираем лишние предупреждения
# import warnings
# warnings.filterwarnings("ignore")

# Настройка для воспроизводимости
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# Подготовка данных для GRU4Rec

def prepare_data_for_gru4rec(clicks, max_len):
    """
    Преобразует данные в последовательности для обучения GRU4Rec.
    """
    sequences = []
    targets = []

    # Группируем по сессиям
    grouped = clicks.groupby("session_id")["item_id"].apply(list)

    for session in grouped:
        for i in range(1, len(session)):
            sequences.append(session[:i])
            targets.append(session[i])

    # Паддинг последовательностей
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")

    return padded_sequences, np.array(targets)

# Параметры
max_len = 10  # Максимальная длина сессии

# 1. Создаем train_vocab и test_vocab на основе item_id
train_vocab = set(train_clicks_filtered["item_id"].unique())
test_vocab = set(test_clicks_filtered["item_id"].unique())

# Создаем маппинг токенов в индексы
token_to_index = {token: idx + 1 for idx, token in enumerate(train_vocab)}  # Сдвиг индексов для mask_zero
token_to_index[0] = 0  # Padding токен

assert len(set(token_to_index.values())) == len(token_to_index), "Индексы токенов не уникальны!"

# 5. Преобразуем item_id в индексы
train_clicks_filtered["item_id"] = train_clicks_filtered["item_id"].map(token_to_index)
test_clicks_filtered["item_id"] = test_clicks_filtered["item_id"].map(token_to_index)

# Подготовка тренировочных данных
train_sequences, train_targets = prepare_data_for_gru4rec(train_clicks_filtered, max_len)
print(f"Train sequences: {train_sequences.shape}, Train targets: {train_targets.shape}")

# Подготовка тестовых данных
test_sequences, test_targets = prepare_data_for_gru4rec(test_clicks_filtered, max_len)
print(f"Test sequences: {test_sequences.shape}, Test targets: {test_targets.shape}")

In [ ]:
def split_sessions_for_evaluation(clicks, max_len):
    """
    Разбивает клики на context и ground truth для оценки.
    """
    test_sessions_split = []
    grouped = clicks.groupby("session_id")["item_id"].apply(list)

    for session_id, session in grouped.items():
        for i in range(1, len(session)):
            context = session[:i]
            ground_truth = [session[i]]
            test_sessions_split.append((session_id, context, ground_truth))

    return test_sessions_split

# Разбиение test_clicks_filtered
test_sessions_split = split_sessions_for_evaluation(test_clicks_filtered, max_len)
print(f"Размер test_sessions_split: {len(test_sessions_split)}")

In [ ]:
from tensorflow.keras.regularizers import l2

# Создание модели GRU4Rec
def build_gru4rec(input_dim, embedding_dim=50, gru_units=100, max_len=20):
    """
    Создаёт модель GRU4Rec.
    """
    gru4rec_model = Sequential([
        tf.keras.layers.Masking(mask_value=0, input_shape=(max_len,)),
        tf.keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim, input_length=max_len, mask_zero=True),
        tf.keras.layers.GRU(
            gru_units,
            activation="swish",
            recurrent_activation="tanh",
            use_bias=True,
            return_sequences=True,
            reset_after=True,
            recurrent_initializer="orthogonal",
            recurrent_dropout=0.0,
            dropout=0.2           # Регуляризация на входе
        ),
        tf.keras.layers.GRU(
            gru_units // 2,
            activation="swish",
            recurrent_activation="sigmoid",
            use_bias=True,
            reset_after=True,
            recurrent_initializer="orthogonal",
            dropout=0.2,
            recurrent_dropout=0.0
        ),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(input_dim, activation="softmax", kernel_regularizer=l2(0.001))
    ])
    optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=1e-4)
    gru4rec_model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_crossentropy", tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10)])

    return gru4rec_model


# Подготовка параметров
embedding_input_dim = len(token_to_index) # Учитываем padding


# Построение модели
gru4rec_model = build_gru4rec(input_dim=embedding_input_dim, embedding_dim=64, gru_units=100)
gru4rec_model.summary()

In [ ]:
# Обучение модели
epochs = 10
batch_size = 128
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)
gru4rec_model.fit(
    train_sequences,
    train_targets,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(test_sequences, test_targets),
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
# Генерация рекомендаций
def generate_recommendations_gru4rec(model, context_sequence, top_k=10):
    """
    Генерирует рекомендации для заданного контекста.
    """
    context_sequence_padded = pad_sequences([context_sequence], maxlen=max_len, padding="pre")
    predictions = model.predict(context_sequence_padded)
    top_k_items = np.argsort(predictions[0])[-top_k:][::-1]
    return top_k_items


In [ ]:
# Обновление test_sessions_split с учетом ограниченной длины контекста
test_sessions_split_updated = []

for session_id, group in test_clicks_filtered.groupby("session_id"):
    items = group["item_id"].tolist()
    for i in range(1, len(items)):
        context = items[max(0, i - max_len):i]  # Ограничиваем длину контекста
        ground_truth = items[i:i+1]
        test_sessions_split_updated.append((session_id, context, ground_truth))

# Проверка корректности test_sessions_split_updated
assert all(len(context) > 0 for _, context, _ in test_sessions_split_updated), "Пустые контексты в test_sessions_split_updated!"
assert all(len(ground_truth) == 1 for _, _, ground_truth in test_sessions_split_updated), "Некорректный ground_truth в test_sessions_split_updated!"

# Преобразование test_buys_filtered
assert set(test_buys_filtered["item_id"]).issubset(token_to_index.keys()), "Есть товары, отсутствующие в token_to_index!"
test_buys_filtered["item_id"] = test_buys_filtered["item_id"].map(token_to_index)

# Проверка на NaN
assert not test_buys_filtered["item_id"].isna().any(), "Есть NaN значения в item_id в test_buys_filtered!"

In [ ]:
def evaluate_gru4rec_without_weights(model, test_sessions_split, test_buys_filtered, top_k=10):
    """
    Оценка GRU4Rec.
    """
    metrics = {"precision": [], "recall": [], "ndcg": [], "revenue": [], "conversion": []}

    for session_id, context, ground_truth in test_sessions_split:

        recommendations = generate_recommendations_gru4rec(model, context, top_k)

        # Рассчитываем метрики
        precision, recall = calculate_precision_recall(recommendations, ground_truth, top_k)
        ndcg = calculate_ndcg(recommendations, ground_truth, top_k)

        # Бизнес-метрики
        ground_truth_buys = test_buys_filtered[test_buys_filtered["session_id"] == session_id]["item_id"].tolist()
        revenue, conversion = calculate_revenue_and_conversion(recommendations, ground_truth_buys, test_buys_filtered, top_k)

        # Сохраняем метрики
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["ndcg"].append(ndcg)
        metrics["revenue"].append(revenue)
        metrics["conversion"].append(conversion)

    # Усредняем метрики
    average_metrics = {metric: np.mean(values) for metric, values in metrics.items()}
    return average_metrics

# Оценка модели
average_metrics_without_weights = evaluate_gru4rec_without_weights(
    gru4rec_model,
    test_sessions_split_updated,
    test_buys_filtered,
    top_k=3
)
print(f"Средние метрики для GRU4Rec k=3: {average_metrics_without_weights}")

average_metrics_without_weights = evaluate_gru4rec_without_weights(
    gru4rec_model,
    test_sessions_split_updated,
    test_buys_filtered,
    top_k=5
)
print(f"Средние метрики для GRU4Rec k=5: {average_metrics_without_weights}")

average_metrics_without_weights = evaluate_gru4rec_without_weights(
    gru4rec_model,
    test_sessions_split_updated,
    test_buys_filtered,
    top_k=10
)
print(f"Средние метрики для GRU4Rec k=10: {average_metrics_without_weights}")